###### Environment initialization

In [2]:
import os
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing import sequence

###### Dataset

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
#or load my own file
from google.calab import files
path_to_file = list(files.upload().keys())[0]

###### Read the content of file

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

In [ ]:
#takea look of first 250 characters in text
print(text[:250])

###### Encoding

In [ ]:
vocab = sorted(set(text))
#creating a mapping from unique characters to indices
char2idx = {u:i for u in enumarate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
#lets look aat how of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

In [ ]:
#And here we'll makeafunction that can convert our numeiricvalues to text
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

###### Creating training example

In [ ]:
#our first step will beto create a streamof characters from our text data
seq_length = 100
exmples_per_epoch = len(text)

#create training example / targets
char_dataset =tf.data.Dataset.from_tensor_slices(text_as_int)

#We can ise batch method to turn this stream of chararcters  into batches of desired length
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
#we need to use thses seq of length 101 and split them into input and output
    
def split_input_target(chunk): #for example: hello
    input_text = chunck[:-1] #hell
    target_text = chunck[1:] #ello
    return input_text, target_text #hell, ello

#let use map to apply the above function to every entry
dataset = sequences.map(split_input_target)        

In [ ]:
for x, y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT")
    print(int_to_text(y))

In [ ]:
#finally we need to make training batches
BATCH_SIZE = 64
VACAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###### Building model

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initialize='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(VACAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

###### Creaing loss function

In [ ]:
for input_exampe_batch, target_examle_batch in dtat.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

In [ ]:
#wecan see that the prdictionis an array of 64 arrays, on for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

In [ ]:
#lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

In [ ]:
#and fnnaly well look at a predictionat the first timestep
time_pred =pred[0]
prnt(len(time_pred))
print(tme_pred)

In [ ]:
sample_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = no.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

In [2]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###### Compiling the model

In [ ]:
model.compile(optimizer='adam', loss=loss)

###### Creating checkpoints

In [3]:
#this will allow us toload our model from a checkpoint and continue training it
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.Modelheckpoint(filepaath=checkpoint_prefix,
                                                       save_weights_only=True)

###### Training

In [ ]:
history = model.fit(data, epochs=40, callbacks=[checkpoint_callback])

###### Loading the model

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
#once the model is finished training we can find the latest checkpoint that stores the models weights using the following line
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
#we can load any checkpoint we want by specifying the exact file to load
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint('./training_checkpoints.ckpt_'+ str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

###### Generating Text

In [ ]:
def 